In [ ]:
!pip install kaggle

In [ ]:
import json
from pathlib import Path
def rtouch(self,mode=0o666,exist_ok=True):
    self.parent.mkdir(parents=True,exist_ok=True)
    self.touch(mode,exist_ok)
setattr(Path,'rtouch',rtouch)

In [ ]:
home_directory=Path('/home/fastai')
kaggle_username=kaggle_fastai
kaggle_key=a_log_hexicode

In [ ]:
p=home_directory/'.kaggle/kaggle.json'
p.rtouch(mode=0o700)

In [ ]:
p.write_text(json.dumps({"username":"moerilily","key":"532db79cad4ec8a1ec23fa33fbfcf775"}))
json.loads(p.read_text())

In [ ]:
import kaggle

In [ ]:
kaggle.api.competition_download_files('state-farm-distracted-driver-detection',path=Path('/home/fast/.fastai/data/distracted-drivers'),quiet=False)

In [ ]:
from fastai2.data.external import *
from fastai2.vision.all import *
from math import isclose
import pandas as pd

In [ ]:
file_extract('/home/fast/.fastai/data/distracted-drivers/state-farm-distracted-driver-detection.zip')

In [ ]:
table=pd.read_csv('/home/fast/.fastai/data/distracted-drivers/driver_imgs_list.csv')
table

In [ ]:
samples=4
valid_subjects=table.groupby('subject').count().sample(n=samples)
while(True):
    try: 
        assert(isclose(valid_subjects.img.sum(),22424*.20,rel_tol=1e-03))
        train_classes=table[~table.subject.isin(valid_subjects.index)].classname.unique()
        valid_classes=table[table.subject.isin(valid_subjects.index)].classname.unique()
        assert((train_classes==valid_classes).all())
        break
    except AssertionError as e:
        valid_subjects=table.groupby('subject').count().sample(n=samples)
print(valid_subjects)
valid_subjects=valid_subjects.index.to_numpy()
table['valid']=table.subject.isin(valid_subjects)

In [ ]:
class DataFrameSplitter:
    def __init__(self,df):
        store_attr(self,'df')
    def __call__(self,o,**kwargs):
        path_df=pd.DataFrame([img.name for img in o],columns=['img'])
        valids=path_df.join(self.df.set_index('img'),on='img',rsuffix='r').valid
        return path_df[~valids].index,path_df[valids].index

In [ ]:
pets.data

In [ ]:
pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(valid_pct=0.001),#DataFrameSplitter(table),
                 get_y=parent_label,
                 item_tfms=Resize(480),
                 batch_tfms=aug_transforms(do_flip=False,size=480, min_scale=0.75))
dls = pets.dataloaders(Path('/home/fast/.fastai/data/distracted-drivers/imgs/train'),verbose=True,bs=32)
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=error_rate,wd=0.1).to_fp16()

In [ ]:
learn.lr_find(num_it=200)

In [ ]:
learn.fit_one_cycle(1,lr_max=0.0075) #ep1 lr0.12 wd0.05 vloss0.29

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5,lr_max=1.3e-05) #5 epochs = 0.35

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.save('big_submission4')

In [ ]:
load_model(Path('/home/fast/fastai2/anime/nbs/models/big_submission4.pth'),learn.model,learn.opt)

In [ ]:
test_dl=dls.test_dl(Path('/home/fast/.fastai/data/distracted-drivers/imgs/test/').ls(),bs=32)

In [ ]:
preds=learn.get_preds(dl=test_dl)

In [ ]:
test_df=pd.DataFrame(preds[0].numpy(),columns=['c'+str(i) for i in range(10)])

In [ ]:
test_df['img']=pd.Series([p.name for p in Path('/home/fast/.fastai/data/distracted-drivers/imgs/test/').ls()])
test_df=test_df.set_index('img')

In [ ]:
test_df.to_csv('seventh_submission.csv')

In [ ]:
kaggle.api.competition_submit('seventh_submission.csv','my seventh submission','state-farm-distracted-driver-detection')

In [ ]:
dls.show_batch(max_n=64)

In [ ]:
test_dl.show_batch(max_n=64)

In [ ]:
!pip install ipywidgets

In [ ]:
from fastai2.vision.widgets import *

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
table[table.classname=='c9'].count()

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(30)

In [ ]:
dls.train.show_batch(max_n=32)

In [ ]:
dls.valid.show_batch(max_n=32)